# FATMA TUĞÇE MUKUL

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from fuzzywuzzy import process
import warnings
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder

warnings.filterwarnings("ignore", category=FutureWarning)

In [15]:

# Adım 1: Süre Sütunlarını Temizleme ve Dönüştürme
def clean_and_convert_duration_columns(df):
    """
    'UygulamaSuresi' ve 'TedaviSuresi' sütunlarındaki metinsel ifadeleri temizler
    ve sayısal veri tipine dönüştürür.

    Args:
        df (pd.DataFrame): İşlenecek DataFrame.

    Returns:
        pd.DataFrame: Temizlenmiş ve dönüştürülmüş sütunlara sahip DataFrame.
    """
    df_copy = df.copy()

    if 'UygulamaSuresi' in df_copy.columns:
        df_copy['UygulamaSuresi'] = df_copy['UygulamaSuresi'].str.replace(' Dakika', '', regex=False).str.strip()
        df_copy['UygulamaSuresi'] = df_copy['UygulamaSuresi'].astype(int)

    if 'TedaviSuresi' in df_copy.columns:
        df_copy['TedaviSuresi'] = df_copy['TedaviSuresi'].str.replace(' Seans', '', regex=False).str.strip()
        df_copy['TedaviSuresi'] = df_copy['TedaviSuresi'].astype(int)

    return df_copy


# Adım 2: Eksik Değerleri Doldurma
def handle_missing_values(df):
    """
    DataFrame'deki eksik değerleri farklı stratejiler kullanarak doldurur.
    - KanGrubu, Alerji, KronikHastalik, Cinsiyet: HastaNo grubuna göre ffill/bfill
    - Bolum, Cinsiyet, Tanilar, UygulamaYerleri: Mod doldurma
    - Yas, KanGrubu (KNNImputer ile)
    - Alerji, KronikHastalik: 'Bilinmiyor' ile doldurma
    """
    df_copy = df.copy()

    # 1. HastaNo grubuna göre ileri ve geri doldurma
    columns_to_ffill_bfill = ['KanGrubu', 'Alerji', 'KronikHastalik', 'Cinsiyet']
    for col in columns_to_ffill_bfill:
        if col in df_copy.columns:
            df_copy[col] = df_copy.groupby('HastaNo')[col].transform(lambda x: x.ffill().bfill())

    # 2. 'Bolum' sütunundaki NaN değerlerini belirli bir değerle doldurma
    if 'Bolum' in df_copy.columns:
        df_copy['Bolum'].fillna('Fiziksel Tıp Ve Rehabilitasyon,Solunum Merkezi', inplace=True)


    # 3. 'Yas' ve 'KanGrubu' için KNNImputer kullanarak doldurma
    if 'Yas' in df_copy.columns and 'Cinsiyet' in df_copy.columns and 'KanGrubu' in df_copy.columns:
        df_temp_knn = df_copy[['Yas', 'Cinsiyet', 'KanGrubu']].copy()

        df_encoded_knn = pd.get_dummies(df_temp_knn, columns=['Cinsiyet', 'KanGrubu'], dummy_na=True)

        if 'KanGrubu_nan' in df_encoded_knn.columns:
            df_encoded_knn['KanGrubu_nan'] = df_encoded_knn['KanGrubu_nan'].replace({1: np.nan, 0: 0})

        imputer = KNNImputer(n_neighbors=5)
        df_filled_knn = pd.DataFrame(imputer.fit_transform(df_encoded_knn), columns=df_encoded_knn.columns, index=df_encoded_knn.index)

        df_copy['Yas'] = df_filled_knn['Yas']

        kan_grubu_cols = [col for col in df_filled_knn.columns if 'KanGrubu_' in col and col != 'KanGrubu_nan']
        if kan_grubu_cols: 
            df_copy['KanGrubu'] = df_filled_knn[kan_grubu_cols].idxmax(axis=1).str.replace('KanGrubu_', '')

    # 4. 'Tanilar' ve 'UygulamaYerleri' sütunlarındaki NaN değerlerini mod ile doldurma
    if 'Tanilar' in df_copy.columns:
        tanilar_mod = df_copy['Tanilar'].mode()[0]
        df_copy['Tanilar'].fillna(tanilar_mod, inplace=True)

    if 'UygulamaYerleri' in df_copy.columns:
        uygulama_yerleri_mod = df_copy['UygulamaYerleri'].mode()[0]
        df_copy['UygulamaYerleri'].fillna(uygulama_yerleri_mod, inplace=True)

    # 5. 'Alerji' ve 'KronikHastalik' sütunlarındaki kalan NaN değerlerini 'Bilinmiyor' ile doldurma
    columns_to_fill_with_unknown = ['Alerji', 'KronikHastalik', 'Cinsiyet']
    for col in columns_to_fill_with_unknown:
        if col in df_copy.columns:
            df_copy[col].fillna('Bilinmiyor', inplace=True)

    return df_copy 


# Adım 3: Metin Temizliği ve Özellik Mühendisliği
def text_cleaning_and_feature_engineering(df):
    """
    'Tanilar', 'TedaviAdi', 'Alerji' ve 'KronikHastalik' sütunlarında
    metin temizliği, normalizasyon ve yeni özellikler oluşturma işlemlerini yapar.
    """
    df_copy = df.copy()

    # --- 'Tanilar' Sütunu İşlemleri ---
    if 'Tanilar' in df_copy.columns:
        def clean_and_split_diagnoses(text):
            if not isinstance(text, str):
                return []
            text = text.lower()
            text = text.strip()
            text = re.sub(r'[,;:-]', ' ', text)
            text = re.sub(r'\s+', ' ', text)
            text = text.replace(' ve ', ' ')
            diagnoses_list = text.split()
            return diagnoses_list

        df_copy['Cleaned_Tanilar'] = df_copy['Tanilar'].apply(clean_and_split_diagnoses)



    # --- 'TedaviAdi' Sütunu İşlemleri ---
    if 'TedaviAdi' in df_copy.columns:
        df_copy['TedaviAdi'] = df_copy['TedaviAdi'].str.lower().str.strip()
        df_copy['TedaviAdi'] = df_copy['TedaviAdi'].str.replace(r'[^a-zğüşöçı\s]', '', regex=True)
        df_copy['TedaviAdi'] = df_copy['TedaviAdi'].str.replace(r'\s+', ' ', regex=True).str.strip()

        tedavi_esleme = {
            'impingement': 'omuz impingement',
            'i̇mpingement': 'omuz impingement',
            'i̇v disk bozukluğu-bel': 'bel fıtığı',
            'iv disk bozukluğu-bel': 'bel fıtığı',
            'iv disk bozukluğu-bel-2': 'bel fıtığı',
            'dorsalji': 'dorsalji',
            'gonartroz-meniskopati': 'gonartroz ve meniskopati',
            'gonartroz-meniskopati + kalkaneal spur': 'gonartroz, meniskopati ve kalkaneal spur',
            'boyun-trapezz': 'boyun ve trapez',
            'dorsalji -boyun+trapez': 'dorsalji, boyun ve trapez',
            'dorsalji-bel+ eklem ağrsıı': 'dorsalji, bel ve eklem ağrısı',
            'omuz el sendromu': 'omuz-el sendromu',
            'alt ekstremite atrofi-bilateral': 'bilateral alt ekstremite atrofisi',
            'deneme': 'bilinmiyor',
            'onur': 'bilinmiyor'
        }
        df_copy['TedaviAdi'] = df_copy['TedaviAdi'].replace(tedavi_esleme, regex=False)

        tedavi_esleme_regex = {
            r'ağrs\b': 'ağrısı',
            r'impingemen\b': 'impingement',
            r'ksalğ\b': 'kısalığı',
            r'reh\b': 'rehabilitasyonu',
            r'boz\b': 'bozukluğu',
            r'krğ\b': 'kırığı',
            r'çkğ\b': 'çıkığı',
            r'op lusu\b': 'operasyonu sonrası',
            r'yrtğ\b': 'yırtığı',
            r'skşmas\b': 'sıkışması',
            r'yaygn\b': 'yaygın',
            r'myodascial ağr\b': 'miyofasiyal ağrı',
            r'bialetarl\b': 'bilateral',
            r'impimgement send\b': 'impingement sendromu',
            r'öçb\b': 'ön çapraz bağ',
            r'iv disk bozukluğu\b': 'servikal disk hastalığı',
            r'iv disk boyun\b': 'servikal disk hastalığı',
            r'fibula krğ\b': 'fibula kırığı',
            r'parmak krğ\b': 'parmak kırığı',
            r'malleol krğ\b': 'malleol kırığı',
            r'kemik uzatma op lusu\b': 'kemik uzatma operasyonu sonrası',
            r'kalça labrum yrtğ op\b': 'kalça labrum yırtığı operasyonu',
            r'boyun ve srt ağrs\b': 'boyun ve sırt ağrısı',
            r'ftğ\b': 'fıtığı',
            r'spinal manüpilasyon\b': 'spinal manipülasyon',
            r'palntar\b': 'plantar',
            r'muscular strain\b': 'musküler strain',
            r'kalça impingiment\b': 'kalça impingement',
            r'tetik parmak operasyonu sonras\b': 'tetik parmak operasyonu sonrası',
            r'\bsrt\b': 'sırt',
            r'\bxx\b': '',
        }
        for pattern, replacement in tedavi_esleme_regex.items():
            df_copy['TedaviAdi'] = df_copy['TedaviAdi'].str.replace(pattern, replacement, regex=True)

        df_copy['TedaviAdi'] = df_copy['TedaviAdi'].str.replace(r'\b(\w+)( \1)+\b', r'\1', regex=True)
        df_copy['TedaviAdi'] = df_copy['TedaviAdi'].str.replace(r'\s+', ' ', regex=True).str.strip()
        df_copy = df_copy[df_copy['TedaviAdi'] != ''] # Boş kalanları at

        def fuzzy_match_and_replace(df_target, column, threshold=85):
            unique_names = df_target[column].unique()
            standardized_names = {}
            for name in unique_names:
                if pd.isna(name):
                    continue
                if name not in standardized_names:
                    matches = process.extractBests(name, unique_names, score_cutoff=threshold)
                    main_name = matches[0][0] if matches else name
                    for match, score in matches:
                        standardized_names[match] = main_name
            return df_target[column].map(standardized_names)

        df_copy['TedaviAdi'] = fuzzy_match_and_replace(df_copy.copy(), 'TedaviAdi', threshold=85)

    # --- 'Alerji' Sütunu İşlemleri ---
    if 'Alerji' in df_copy.columns:
        def clean_allergies(s):
            if not isinstance(s, str):
                return 'Yok'
            s = s.lower().strip()
            if s == 'bilinmiyor':
                return 'Yok'
            allergens = [a.strip() for a in s.split(',')]
            mapping = {
                'volteren': 'voltaren',
                'gri̇pi̇n': 'gripin',
                'yer fıstığı': 'yer fıstığı'
            }
            standardized_allergens = [mapping.get(a, a) for a in allergens]
            unique_allergens = sorted(list(set(standardized_allergens)))
            return ', '.join(unique_allergens)

        df_copy['Alerji'] = df_copy['Alerji'].apply(clean_allergies)

    # --- 'KronikHastalik' Sütunu İşlemleri ---
    if 'KronikHastalik' in df_copy.columns:
        def clean_chronic_diseases(text):
            if not isinstance(text, str):
                return text
            text = text.lower()
            text = text.replace('bilinmiyor', 'yok')
            text = re.sub(r'[,;:]', ' ', text)
            text = re.sub(r'\s+', ' ', text)
            return text.strip()

        df_copy['KronikHastalik_Cleaned'] = df_copy['KronikHastalik'].apply(clean_chronic_diseases)


    # --- Gereksiz Sütunları Silme ---
    gereksiz_sutunlar = [
        'HastaNo', # Bu sütunu düşürmeden önce dikkatli olun, Patient ID olabilir
        'KronikHastalik',
        'Tanilar',
    ]
    sutunlar_to_drop = [col for col in gereksiz_sutunlar if col in df_copy.columns]
    if sutunlar_to_drop:
        df_copy.drop(columns=sutunlar_to_drop, inplace=True)

    return df_copy



# Adım 4: Kategorik Değişkenleri Encode Etme
from sklearn.preprocessing import MultiLabelBinarizer
import pandas as pd

def encode_categorical_features(df):
    """
    Kategorik özellikleri One-Hot Encoding kullanarak
    sayısal formata dönüştürür.
    """
    df_copy = df.copy()

    # --- 'Cinsiyet' Sütunu için One-Hot Encoding ---
    if 'Cinsiyet' in df_copy.columns:
        cinsiyet_encoded = pd.get_dummies(df_copy['Cinsiyet'], prefix='Cinsiyet')
        df_copy = pd.concat([df_copy.drop('Cinsiyet', axis=1), cinsiyet_encoded], axis=1)

    # --- 'Alerji' Sütunu için One-Hot Encoding (MultiLabelBinarizer ile) ---
    if 'Alerji' in df_copy.columns:
        alerji_listeleri = df_copy['Alerji'].str.split(', ')
        alerji_listeleri = [item if item != ['Yok'] else [] for item in alerji_listeleri]

        mlb = MultiLabelBinarizer()
        encoded_alerji = pd.DataFrame(mlb.fit_transform(alerji_listeleri),
                                      columns=[f'Alerji_{col}' for col in mlb.classes_],
                                      index=df_copy.index)
        
        # Orijinal DataFrame ile birleştirme ve 'Alerji' sütununu silme
        df_copy = pd.concat([df_copy.drop('Alerji', axis=1), encoded_alerji], axis=1)

    return df_copy




# Adım 5: Görselleştirmeler ve Son Kontroller
def perform_visualizations_and_final_checks(df):
    """
    Veri seti üzerinde çeşitli görselleştirmeler yapar ve son kontrolleri (örn. yinelenen sütunları düşürme) gerçekleştirir.
    Görseller ekranda gösterilmez.
    """
    df_copy = df.copy()


    # --- 1. Yinelenen Sütunları Düşürme ---
    duplicates = df_copy.columns[df_copy.columns.duplicated()]
    if len(duplicates) > 0:
        print(f"Uyarı: {len(duplicates)} adet yinelenen sütun bulundu ve düşürüldü: {list(duplicates)}")
        df_copy = df_copy.loc[:, ~df_copy.columns.duplicated()]

    # --- 2. Yaş ve Tedavi Süresi İlişkisi ---
    if 'Yas' in df_copy.columns and 'TedaviSuresi' in df_copy.columns:
        plt.figure(figsize=(10,6))
        sns.scatterplot(x='Yas', y='TedaviSuresi', data=df_copy, alpha=0.6)
        plt.title('Yaş ve Tedavi Süresi İlişkisi')
        plt.xlabel('Yaş')
        plt.ylabel('Tedavi Süresi')
        # plt.show() # Kullanıcıya gösterilmeyeceği için devre dışı bırakıldı
        plt.close()

    # --- 3. Bölümlere Göre Ortalama Tedavi Süresi ---
    if 'Bolum' in df_copy.columns and 'TedaviSuresi' in df_copy.columns:
        plt.figure(figsize=(12,6))
        sns.barplot(x='Bolum', y='TedaviSuresi', data=df_copy, ci=None, palette='Blues')
        plt.title('Bölümlere Göre Ortalama Tedavi Süresi')
        plt.xlabel('Bölüm')
        plt.ylabel('Ortalama Tedavi Süresi')
        plt.xticks(rotation=45)
        # plt.show() # Kullanıcıya gösterilmeyeceği için devre dışı bırakıldı
        plt.close()
        
    return df_copy



In [16]:
# --- Ana EDA Pipeline Fonksiyonu ---
def run_eda_pipeline(df, output_filename="temizlenmis_veri.csv"):
    """
    Verilen DataFrame üzerinde EDA pipeline'ını çalıştırır ve temizlenmiş veriyi kaydeder.

    Args:
        df (pd.DataFrame): İşlenecek orijinal DataFrame.
        output_filename (str): Temizlenmiş verinin kaydedileceği dosya adı (varsayılan: "temizlenmis_veri.csv").

    Returns:
        pd.DataFrame: Temizlenmiş ve işlenmiş DataFrame.
    """
    df_processed = df.copy()

    print("EDA Pipeline Başlatılıyor...")

    # Adım 1: Süre sütunlarını temizleme
    print("Adım 1/6: Süre sütunları temizleniyor ve dönüştürülüyor...")
    df_processed = clean_and_convert_duration_columns(df_processed)
    print("Adım 1 tamamlandı.")

    # Adım 2: Eksik değerleri doldurma
    print("Adım 2/6: Eksik değerler dolduruluyor...")
    df_processed = handle_missing_values(df_processed)
    print("Adım 2 tamamlandı.")

    # Adım 3: Metin Temizliği ve Özellik Mühendisliği
    print("Adım 3/6: Metin sütunları temizleniyor ve yeni özellikler oluşturuluyor...")
    df_processed = text_cleaning_and_feature_engineering(df_processed)
    print("Adım 3 tamamlandı.")

    # Adım 4: Kategorik değişkenleri encode etme
    print("Adım 4/6: Kategorik değişkenler encode ediliyor...")
    df_processed = encode_categorical_features(df_processed)
    print("Adım 4 tamamlandı.")

    # Adım 5: Görselleştirmeler ve Son Kontroller
    print("Adım 5/6: Görselleştirmeler yapılıyor ve son kontroller gerçekleştiriliyor...")
    df_processed = perform_visualizations_and_final_checks(df_processed)
    print("Adım 5 tamamlandı.")

    # Adım 6: Temizlenmiş veri setini kaydetme
    print(f"Adım 6/6: Temizlenmiş veri '{output_filename}' olarak kaydediliyor...")
    df_processed.to_csv(output_filename, index=False)
    print(f"Adım 6 tamamlandı: Veri başarıyla '{output_filename}' olarak kaydedildi.")
    print("EDA Pipeline Tamamlandı!")

    return df_processed

In [17]:
df = pd.read_excel("talent.xlsx")
processed_df = run_eda_pipeline(df, output_filename="temizlenmis_veri_son.csv")


EDA Pipeline Başlatılıyor...
Adım 1/6: Süre sütunları temizleniyor ve dönüştürülüyor...
Adım 1 tamamlandı.
Adım 2/6: Eksik değerler dolduruluyor...
Adım 2 tamamlandı.
Adım 3/6: Metin sütunları temizleniyor ve yeni özellikler oluşturuluyor...
Adım 3 tamamlandı.
Adım 4/6: Kategorik değişkenler encode ediliyor...
Adım 4 tamamlandı.
Adım 5/6: Görselleştirmeler yapılıyor ve son kontroller gerçekleştiriliyor...
Adım 5 tamamlandı.
Adım 6/6: Temizlenmiş veri 'temizlenmis_veri_son.csv' olarak kaydediliyor...
Adım 6 tamamlandı: Veri başarıyla 'temizlenmis_veri_son.csv' olarak kaydedildi.
EDA Pipeline Tamamlandı!


In [21]:
print("\nİşlenmiş DataFrame'in İlk 10 Satırı:")
processed_df.head(10)


İşlenmiş DataFrame'in İlk 10 Satırı:


,Yas,KanGrubu,Uyruk,Bolum,TedaviAdi,TedaviSuresi,UygulamaYerleri,UygulamaSuresi,Cleaned_Tanilar,KronikHastalik_Cleaned,...,Cinsiyet_Kadın,Alerji_arveles,Alerji_coraspin,Alerji_gripin,Alerji_novalgin,Alerji_polen,Alerji_sucuk,Alerji_toz,Alerji_voltaren,Alerji_yer fıstığı
0,60.0,0 Rh+,Türkiye,"Fiziksel Tıp Ve Rehabilitasyon,Solunum Merkezi",el bileği tendinitganglion,5,Ayak Bileği,20,"[ayak, bileği, ayağın, yüzeysel, yaralanması]",becker musküler distrofisi hiportiroidizm kalp...,...,True,0,0,0,0,0,0,1,0,0
1,28.0,0 Rh+,Türkiye,"Fiziksel Tıp Ve Rehabilitasyon,Solunum Merkezi",dorsalji boyuntrapezskapular,15,Boyun,20,"[omuzun, darbe, sendromu, dorsalji̇, di̇ğer, s...",duchenne musküler distrofisi myastenia gravis ...,...,False,0,0,0,0,0,0,0,0,0
2,28.0,0 Rh+,Türkiye,"Fiziksel Tıp Ve Rehabilitasyon,Solunum Merkezi",dorsalji boyuntrapezskapular,15,"Boyun,Sırt",20,"[omuzun, darbe, sendromu, dorsalji̇, di̇ğer, s...",duchenne musküler distrofisi myastenia gravis ...,...,False,0,0,0,0,0,0,0,0,0
3,28.0,0 Rh+,Türkiye,"Fiziksel Tıp Ve Rehabilitasyon,Solunum Merkezi",dorsalji boyuntrapezskapular,15,Boyun,5,"[omuzun, darbe, sendromu, dorsalji̇, di̇ğer, s...",duchenne musküler distrofisi myastenia gravis ...,...,False,0,0,0,0,0,0,0,0,0
4,28.0,0 Rh+,Türkiye,"Fiziksel Tıp Ve Rehabilitasyon,Solunum Merkezi",dorsalji boyuntrapezskapular,15,"Boyun,Sırt",20,"[omuzun, darbe, sendromu, dorsalji̇, di̇ğer, s...",duchenne musküler distrofisi myastenia gravis ...,...,False,0,0,0,0,0,0,0,0,0
5,28.0,0 Rh+,Türkiye,"Fiziksel Tıp Ve Rehabilitasyon,Solunum Merkezi",dorsalji boyuntrapezskapular,15,Boyun,20,"[omuzun, darbe, sendromu, dorsalji̇, di̇ğer, s...",duchenne musküler distrofisi myastenia gravis ...,...,False,0,0,0,0,0,0,0,0,0
6,60.0,0 Rh+,Türkiye,"Fiziksel Tıp Ve Rehabilitasyon,Solunum Merkezi",parapleji,10,Bel,30,"[parapleji, tetrapleji]",yok,...,False,0,0,0,1,1,0,0,0,0
7,60.0,0 Rh+,Türkiye,"Fiziksel Tıp Ve Rehabilitasyon,Solunum Merkezi",parapleji,10,Bel,20,"[parapleji, tetrapleji]",yok,...,False,0,0,0,1,1,0,0,0,0
8,60.0,0 Rh+,Türkiye,"Fiziksel Tıp Ve Rehabilitasyon,Solunum Merkezi",parapleji,10,Bel,20,"[parapleji, tetrapleji]",yok,...,False,0,0,0,1,1,0,0,0,0
9,65.0,0 Rh+,Türkiye,"Fiziksel Tıp Ve Rehabilitasyon,Solunum Merkezi",volar plak rehabilitasyonu,15,Sol El Bilek Bölgesi,15,"[artroz, tanımlanmamış, el]",hiportiroidizm diyabet duchenne musküler distr...,...,True,0,0,0,1,0,0,0,0,0
